In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)

from tqdm import tqdm
import random

# Goal

The June edition of the 2022 Tabular Playground series is all about data imputation. The dataset has similarities to the May 2022 Tabular Playground, except that there are no targets. Rather, there are missing data values in the dataset, and your task is to predict what these values should be.

For this challenge, you are given (simulated) manufacturing control data that contains missing values due to electronic errors. Your task is to predict the values of all missing data in this dataset. (Note, while there are continuous and categorical features, only the continuous features have missing values.)

In [2]:
data = pd.read_csv('../input/tabular-playground-series-jun-2022/data.csv', index_col = 'row_id')
submission = pd.read_csv('../input/tabular-playground-series-jun-2022/sample_submission.csv', index_col = 'row-col')

print(data.shape)
print(submission.shape)

(1000000, 80)
(1000000, 1)


In [3]:
submission.head()

,value
row-col,
0-F_1_14,0.0
0-F_3_23,0.0
1-F_3_24,0.0
2-F_1_2,0.0
2-F_4_2,0.0


In [4]:
data.head(3)

,F_1_0,F_1_1,F_1_2,F_1_3,F_1_4,F_1_5,F_1_6,F_1_7,F_1_8,F_1_9,F_1_10,F_1_11,F_1_12,F_1_13,F_1_14,F_2_0,F_2_1,F_2_2,F_2_3,F_2_4,F_2_5,F_2_6,F_2_7,F_2_8,F_2_9,F_2_10,F_2_11,F_2_12,F_2_13,F_2_14,F_2_15,F_2_16,F_2_17,F_2_18,F_2_19,F_2_20,F_2_21,F_2_22,F_2_23,F_2_24,F_3_0,F_3_1,F_3_2,F_3_3,F_3_4,F_3_5,F_3_6,F_3_7,F_3_8,F_3_9,F_3_10,F_3_11,F_3_12,F_3_13,F_3_14,F_3_15,F_3_16,F_3_17,F_3_18,F_3_19,F_3_20,F_3_21,F_3_22,F_3_23,F_3_24,F_4_0,F_4_1,F_4_2,F_4_3,F_4_4,F_4_5,F_4_6,F_4_7,F_4_8,F_4_9,F_4_10,F_4_11,F_4_12,F_4_13,F_4_14
row_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,-0.354591,-0.464038,2.304115,0.734486,1.696395,0.136285,-0.518344,0.502640,-1.852504,-0.500665,-1.416075,1.201521,0.551902,-0.759827,NaN,2,2,0,3,2,1,1,3,2,2,3,3,2,5,4,1,0,1,0,2,1,2,0,1,2,-0.240522,0.061529,0.561090,-0.171943,-0.431996,0.473508,0.596924,0.819306,1.479061,1.264616,-1.116881,0.759443,-0.086915,-0.620685,0.057216,1.076380,-0.780608,-1.940907,-0.717021,0.599093,0.498347,0.118770,-0.228913,NaN,0.301610,5.547214,1.066871,-0.134313,-0.101040,-0.660871,3.744152,0.794438,0.265185,-0.561809,0.196480,0.373434,6.206995,3.809505,1.236486,1.182055
1,1.380940,-0.499626,-0.418548,1.911725,-0.826130,-1.715371,-0.577091,-1.041486,0.596067,-0.363425,-0.853630,0.674525,0.843058,-0.041438,0.259496,3,5,2,1,4,2,3,2,0,0,9,3,1,4,1,2,1,0,1,1,1,1,3,2,5,-0.446068,1.433358,0.040099,-1.994062,-0.602324,-0.611391,-1.151884,0.065485,0.352023,-0.843751,1.167272,0.921445,-0.839827,0.759015,-1.547387,0.720435,-0.944045,1.796462,-1.046357,-0.581515,0.704543,0.375222,0.705963,0.032771,NaN,-1.707374,-1.188114,-0.562419,-1.462988,1.290672,-2.895826,-0.738275,2.361818,-0.060753,0.727249,-0.271882,5.232157,-4.218259,-2.724883,-0.063775
2,0.256023,-1.059874,NaN,0.345678,1.513814,1.243864,-0.509648,-0.800481,-0.115945,0.595777,-0.073235,-1.381605,-0.108676,0.703693,-0.464042,3,2,1,3,2,1,0,6,1,1,3,4,1,2,2,1,2,2,4,8,1,5,1,2,4,-0.770592,0.483139,-0.636484,-1.305018,-2.089889,0.276761,-1.208640,-0.855769,0.232363,0.215841,-1.031405,0.582437,-0.314639,-0.497409,0.489356,0.915049,-0.513410,0.904206,-0.056089,0.212927,-0.574126,-1.517749,-0.888472,0.142264,1.000822,1.914908,3.877128,NaN,0.358635,0.443973,2.252834,0.472496,2.491386,0.353381,-0.260682,-0.000833,-0.116457,-2.131747,3.661499,-0.131576


# sklearn.impute.IterativeImputer
- Multivariate imputer that estimates each feature from all the others.

A strategy for imputing missing values by modeling each feature with missing values as a function of other features in a round-robin fashion.

In [5]:
import lightgbm
import xgboost
import catboost

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [6]:
imp = IterativeImputer(
    estimator=xgboost.XGBRegressor(
        n_estimators=100,
        random_state=123,
        tree_method='gpu_hist',
    ),
    missing_values=np.nan,
    max_iter=6,
    initial_strategy='mean',
    imputation_order='ascending',
    verbose=2,
    random_state=123
)

data[:] = imp.fit_transform(data)

[IterativeImputer] Completing matrix with shape (1000000, 80)
[IterativeImputer] Ending imputation round 1/6, elapsed time 331.31
[IterativeImputer] Change: 23.49721808848335, scaled tolerance: 0.031229363 
[IterativeImputer] Ending imputation round 2/6, elapsed time 661.29
[IterativeImputer] Change: 7.170091152191162, scaled tolerance: 0.031229363 
[IterativeImputer] Ending imputation round 3/6, elapsed time 992.47
[IterativeImputer] Change: 6.673506701365113, scaled tolerance: 0.031229363 
[IterativeImputer] Ending imputation round 4/6, elapsed time 1322.32
[IterativeImputer] Change: 5.0871241837739944, scaled tolerance: 0.031229363 
[IterativeImputer] Ending imputation round 5/6, elapsed time 1651.82
[IterativeImputer] Change: 5.503079295158386, scaled tolerance: 0.031229363 
[IterativeImputer] Ending imputation round 6/6, elapsed time 1982.04
[IterativeImputer] Change: 5.2916890676133335, scaled tolerance: 0.031229363 


In [7]:
for i in tqdm(submission.index):
    row = int(i.split('-')[0])
    col = i.split('-')[1]
    submission.loc[i, 'value'] = data.loc[row, col]

submission.to_csv("submission.csv")
submission

100%|██████████| 1000000/1000000 [01:30<00:00, 11098.39it/s]


,value
row-col,
0-F_1_14,0.029139
0-F_3_23,-0.149457
1-F_3_24,-0.015307
2-F_1_2,0.155732
2-F_4_2,0.244066
...,...
999993-F_4_2,-0.187566
999994-F_3_10,-0.070625
999994-F_4_9,-0.070952
